# 将爬取的文本数据处理为ChatGLM3对话格式
1. 获取文本：爬取的文本数据存储在MySQL中，连接MySQL获取文本数据，Text(id, title, content, section)
2. 构造数据集：将section和title拼接起来作为prompt，划分训练集(80%)、验证集(20%)、测试集(20%)
3. 格式处理：将数据集处理为ChatGLM3对话格式，保存为json文件

## 1. 获取文本
爬取的文本数据存储在MySQL中，连接MySQL获取文本数据，Text(id, title, content, section)

In [1]:
import pymysql

In [2]:
HOST = 'localhost'
USER = 'root'
PASSWORD = '020411'
DATABASE = 'maxdb'

In [3]:
connection = pymysql.connect(host=HOST, user=USER, password=PASSWORD, db=DATABASE)
cursor = connection.cursor()

In [4]:
with connection, cursor:
    cursor.execute('select title, content, section from text;')
    results = cursor.fetchall()
len(results)

3051

## 2. 构造数据集
将section和title拼接起来作为prompt，划分训练集(80%)、验证集(20%)、测试集(20%)


In [5]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

In [6]:
data = pd.DataFrame(results, columns=['title', 'content', 'section'])
data.head()

,title,content,section
0,关于申报2023年河北省研究生创新资助项目的通知,院属相关研究生及导师：根据学校《关于申报2023年河北省研究生创新资助项目的通知》，现将我院...,通知公告
1,关于机械工程学院2021级本科生专业分流绩点加分的公示,根据《机械工程学院2021级大类招生专业分流方案》相关要求，通过学生申报，学院核实，现对20...,通知公告
2,关于机械工程学院2021级大类招生专业分流结果的公示,2021级机械类专业本科生、院属各单位：根据《机械工程学院2021级大类招生专业分流方案》的...,通知公告
3,关于评选燕山大学2020/2021学年度优秀博士硕士学位论文暨推荐河北省优秀博士硕士学位论文的通知,院属各系部、研究生导师及相关研究生：学位论文质量是衡量研究生培养质量的重要标准。为进一步鼓励...,通知公告
4,关于2022年度“俄乌白项目”推荐出国访学人员的公示,在2022年度国家留学基金管理委员会资助的促进与俄罗斯、乌克兰、白俄罗斯国际合作培养项目(简...,通知公告


In [17]:
fig = px.bar(data['section'].value_counts(), y='count', orientation='v', title='各板块/章节记录数统计')
fig.show()

In [7]:
for idx, row in data.iterrows():
    title = f"{row['section']}#{row['title']}"
    data.loc[idx, 'title'] = title
data.drop(columns=['section'], inplace=True)
data.head()

,title,content
0,通知公告#关于申报2023年河北省研究生创新资助项目的通知,院属相关研究生及导师：根据学校《关于申报2023年河北省研究生创新资助项目的通知》，现将我院...
1,通知公告#关于机械工程学院2021级本科生专业分流绩点加分的公示,根据《机械工程学院2021级大类招生专业分流方案》相关要求，通过学生申报，学院核实，现对20...
2,通知公告#关于机械工程学院2021级大类招生专业分流结果的公示,2021级机械类专业本科生、院属各单位：根据《机械工程学院2021级大类招生专业分流方案》的...
3,通知公告#关于评选燕山大学2020/2021学年度优秀博士硕士学位论文暨推荐河北省优秀博士硕...,院属各系部、研究生导师及相关研究生：学位论文质量是衡量研究生培养质量的重要标准。为进一步鼓励...
4,通知公告#关于2022年度“俄乌白项目”推荐出国访学人员的公示,在2022年度国家留学基金管理委员会资助的促进与俄罗斯、乌克兰、白俄罗斯国际合作培养项目(简...


In [8]:
data = data.sample(frac=1).reset_index(drop=True)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=200)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=200)
len(train_data), len(valid_data), len(test_data)

(1830, 610, 611)

In [9]:
train_data.head()

,title,content
290,党群新闻#学院召开主题教育第五次工作推进会,11月25日下午，机械工程学院在机械馆301会议室召开了主题教育第五次工作推进会。学院党委书...
71,通知公告#关于机械工程学院2023年度科研经费专项博士申请的通知,各位博士研究生招生导师：学校已发布燕山大学2023年“科研经费博士研究生专项招生计划”招生简...
1959,党群新闻#学院举办十九大新党章学习辅导报告,12月7日上午10:00，学院在机械馆J3报告厅举办了学习十九大新党章辅导报告，会议邀请马克...
700,党群新闻#学院团委举办团干部培训活动,碧海蓝天秀燕园，春风拂面暖学子！四月二十八日，在这个春暖花开的日子里，在建校90周年和08级...
1240,通知公告#关于分配2023年春季学期研究生业务经费的通知,院属各系部及相关研究生导师：根据研究生院《关于核对2023年春季学期研究生业务经费的通知》要...


## 3. 格式处理
将数据集处理为ChatGLM3对话格式，保存为json文件

使用仅微调对话能力的格式，ChatGLM3对话格式如下
```json
[
  // 一个json对象为一个样本
  {
    "conversations": [
      {
        "role": "system",
        "content": "<system prompt text>"
      },
      {
        "role": "user",
        "content": "<user prompt text>"
      },
      {
        "role": "assistant",
        "content": "<assistant response text>"
      }
    ]
  },
  // ...
]
```

In [10]:
import json
from pathlib import Path

In [11]:
train_file = Path('data/train.json')
valid_file = Path('data/valid.json')
test_file = Path('data/test.json')

In [12]:
def transform_and_save(data, path):
    conversations = []
    for _, row in data.iterrows():
        title = row['title']
        content = row['content']
        conversation = {
            'conversations': [
                {'role': 'user', 'content': title},
                {'role': 'assistant', 'content': content}
            ]
        }
        conversations.append(conversation)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(conversations, f, ensure_ascii=False, indent=4)

In [13]:
transform_and_save(train_data, train_file)
transform_and_save(valid_data, valid_file)
transform_and_save(test_data, test_file)